In [1]:
!pip install openai neo4j

In [3]:
import openai

# OpenAI API 설정
openai_client = openai.Client(api_key="sk")

# Neo4j 연결 설정
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = " " # Neo4j 패스워드 입력

In [4]:
from neo4j import GraphDatabase # Neo4j와 연결할 수 있는 드라이버

# Neo4j 드라이버 설정
class Neo4jDatabase: 
    # Neo4j 데이터베이스에 연결
    def __init__(self, uri, user, password): 
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    # Neo4j 연결을 닫는 함수
    def close(self): 
        self.driver.close()

    # Cypher 쿼리를 실행하는 함수
    def run_query(self, query, parameters={}):
        # Neo4j 세션을 열어 데이터베이스와 통신
        with self.driver.session() as session: 
            try:
                result = session.run(query, parameters) # Cypher 쿼리를 실행하고 결과를 가져옴
                return [record for record in result]
            except Exception as e:
                return f"❌ Neo4j 오류 발생: {str(e)}"

# Neo4j 데이터베이스 객체 생성
db = Neo4jDatabase(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

In [5]:
# 사용자의 질문 (question)을 입력받아 Cypher 쿼리로 변환하는 함수
def generate_cypher_query(question):
    prompt = f"""
    당신은 자연어 질문을 Neo4j 데이터베이스에서 사용할 Cypher 쿼리로 변환하는 AI 어시스턴트입니다.
    이 데이터베이스에는 자동차 제조업체, 브랜드 및 가격에 대한 정보가 포함되어 있습니다.

    다음은 몇 가지 예제입니다:
    - "현대에서 만든 자동차 브랜드는 무엇인가요?" -> MATCH (m:Manufacturer {{Name: '현대'}})-[:PRODUCES]->(b:Brand) RETURN b.Name
    - "람보르기니의 자동차 모델은?" -> MATCH (m:Manufacturer {{Name: '람보르기니'}})-[:PRODUCES]->(b:Brand) RETURN b.Name
    - "벤츠에서 가장 비싼 모델은?" -> MATCH (m:Manufacturer {{Name: '벤츠'}})-[:PRODUCES]->(b:Brand)-[:HAS_PRICE]->(p:Price) RETURN b.Name ORDER BY p.Amount DESC LIMIT 1
    - "1억 원 이상 하는 자동차는?" -> MATCH (b:Brand)-[:HAS_PRICE]->(p:Price) WHERE p.Amount >= 100000000 RETURN b.Name, p.Amount

    다음 질문을 Cypher 쿼리로 변환하세요:
    "{question}"
    """
    
    # GPT-4에게 질문을 전달하여 Cypher 쿼리를 생성하도록 요청
    response = openai_client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": prompt}]
    )
    
    # LLM 응답 정리하되 불필요한 따옴표 제거
    cypher_query = response.choices[0].message.content.strip().strip('"')
    
    print(f"생성된 Cypher 쿼리: {cypher_query}") 
    return cypher_query

In [6]:
# 사용자의 자연어 질문(question)을 입력받아 자동차 데이터를 조회하는 함수
def get_car_data(question):
    # 자연어 질문을 Cypher 쿼리로 변환
    cypher_query = generate_cypher_query(question)
    
    # Cypher 쿼리 실행
    results = db.run_query(cypher_query)
    
    # 검색 결과 반환
    if results:
        return [dict(record) for record in results]
    else:
        return "해당 정보를 찾을 수 없습니다."

In [7]:
user_question = "현대에서 만든 자동차 브랜드는 무엇인가요?"
response = get_car_data(user_question)
print(response)

생성된 Cypher 쿼리: MATCH (m:Manufacturer {Name: '현대'})-[:PRODUCES]->(b:Brand) RETURN b.Name
[{'b.Name': '제네시스'}, {'b.Name': '쏘나타'}, {'b.Name': '아반떼'}, {'b.Name': '그랜저'}]


In [8]:
user_question = "람보르기니의 자동차 모델은?"
response = get_car_data(user_question)
print(response)

생성된 Cypher 쿼리: MATCH (m:Manufacturer {Name: '람보르기니'})-[:PRODUCES]->(b:Brand) RETURN b.Name
[{'b.Name': 'Aventador'}, {'b.Name': 'Huracan'}, {'b.Name': 'Urus'}]


In [9]:
user_question = "1억 원 이상 하는 자동차는?"
response = get_car_data(user_question)
print(response)

생성된 Cypher 쿼리: MATCH (b:Brand)-[:HAS_PRICE]->(p:Price) WHERE p.Amount >= 100000000 RETURN b.Name, p.Amount
[{'b.Name': 'S-Class', 'p.Amount': 150000000}, {'b.Name': '7 Series', 'p.Amount': 130000000}, {'b.Name': '488 GTB', 'p.Amount': 400000000}, {'b.Name': 'Huracan', 'p.Amount': 400000000}, {'b.Name': 'Roma', 'p.Amount': 350000000}, {'b.Name': 'F8 Tributo', 'p.Amount': 450000000}, {'b.Name': 'SF90', 'p.Amount': 600000000}, {'b.Name': 'Model S', 'p.Amount': 110000000}, {'b.Name': 'Model X', 'p.Amount': 120000000}, {'b.Name': 'Aventador', 'p.Amount': 500000000}, {'b.Name': 'Urus', 'p.Amount': 300000000}]
